# Install 

Install this project's requirements. 

In [ ]:
%pip install -r ../requirements.txt


# Configure

Configure the credentials to run the search.

In [ ]:
import ipywidgets as widgets

openai_input = widgets.Text(
    placeholder='Enter your OpenAI API key',
    description='OPENAI_API_KEY:',
)

tavily_input = widgets.Text(
    placeholder='Enter your Tavily API key',
    description='TAVILY_API_KEY:',
)

# Display the widgets.
display(openai_input)
display(tavily_input)


Create the model, search tool, and agent.

In [ ]:
from agent_web_search.agents import create_search_agent
from agent_web_search.models import OpenAIConfig, create_model
from agent_web_search.tools import SearchToolConfig, create_search_tool

# Validate the input values.
if not openai_input.value or not openai_input.value.strip():
    raise ValueError("OpenAI API key cannot be empty or contain only whitespace")

if not tavily_input.value or not tavily_input.value.strip():
    raise ValueError("Tavily API key cannot be empty or contain only whitespace")

# Create the model.
openai_config = OpenAIConfig(
    model_id="gpt-4o",
    api_key=openai_input.value,
)
model = create_model(openai_config)

# Create the search tool.
search_tool_config = SearchToolConfig(api_key=tavily_input.value)
search_tool = create_search_tool(search_tool_config)

# Create the agent.
graph = create_search_agent(model, search_tool)


# Run 

- Import the search agent graph.
- Create the UI.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget.
message_input = widgets.Text(
    value="What's the latest news about AI?",
    placeholder='Type your human message here',
    description='Message:',
    disabled=False
)

# Display the widget.
display(message_input)


Run the search.

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content=message_input.value)]

result = graph.invoke({"messages": messages})
print(result)


Print the results in a friendly format.

In [ ]:
from pprint import pprint

from langchain_core.messages import ToolMessage

# Get only tool messages from the result.
tool_messages = [
    message for message in result["messages"] if isinstance(message, ToolMessage)
]

# Print each tool message (most likely there will be just one) in a readable format.
for i, message in enumerate(tool_messages):
    print(f"\n--- Tool Message {i+1} ---")
    print(f"Tool Name: {message.name}")

    items = message.artifact.get("results", [])

    for j, item in enumerate(items):
        title = item["title"]
        url = item["url"]
        body = item["content"]

        print(f"\n--- Result {j+1} ---")
        print(f"Title: {title}")
        print(f"URL: {url}")
        pprint(body)
